In [ ]:
import multiprocessing
import math
from wiki_dump_reader import Cleaner , iterate
from tqdm import tqdm
import numpy as np
import multiprocessing
import nltk
from collections import Counter, defaultdict
from nltk import ngrams, word_tokenize
nltk.download('punkt')



: 

In [2]:
import time

def runtime(func):
	def wrapper(*args, **kwargs):
		start_time = time.time()
		result = func(*args, **kwargs)
		end_time = time.time()
		print(f"Time taken by {func.__name__}: {end_time - start_time} seconds")
		return result
	return wrapper



In [3]:
def memory_location(func):
	def wrapper(*args, **kwargs):
		print(f"Location of {func.__name__}: {id(func)}")
		return func(*args, **kwargs)
	return wrapper


In [4]:
@runtime
@memory_location
def create_corpus():
	corpus_file = 'Hindi_Corpus.txt'
	corpus_limit = 232729
	page_count = 0
	cleaner = Cleaner()  # Assuming Cleaner class is imported or defined elsewhere.
	
	# Open the output file for writing
	with open(corpus_file, 'w', encoding='utf-8') as output:
		bar = tqdm(total=corpus_limit)  # Initialize the progress bar
		for title, text in iterate('hiwiki-latest-pages-articles.xml'):  
			text = cleaner.clean_text(text)
			cleaned_text, _ = cleaner.build_links(text)
			output.write(title + '\n' + cleaned_text + '\n')
			page_count += 1
			if page_count % 1000 == 0:
				bar.update(1000)  # Update progress bar every 1000 pages
		bar.close() 
		output.close()  # Close the output file
	print(f"\nPage count = {page_count}")  # Print total page count after processing



In [5]:
def create_stop_words():
    """
    Create a list of stop words from a file named 'stopwords.txt'.

    This function reads the stop words from the file, strips newline characters,
    and splits the words by spaces to create a list of stop words.

    Args:
        None
    
    Returns:
        list: A list of stop words.
    
    Raises:
        None
    
    """
    with open('stopwords.txt', 'r', encoding='utf-8') as stop:
        y = stop.readlines()
    x = []
    for element in y:
        element = element.strip('\n')
        x.extend(element.split(' '))
    return x


stop_words=create_stop_words()

def remove_stop_words(string):
    """
    Remove stop words from a given string.

    This function takes a string as input, splits it into words, and removes
    any words that are found in a predefined list of stop words. It then joins
    the remaining words back into a single string and returns it.

    Args:
        string (str): The input string from which stop words are to be removed.

    Returns:
        str: The input string with stop words removed.
    
    Raises:
        None
    
    """
    l = string.split()  # Split the input string into a list of words
    return_list = []
    for x in l:
        if x not in stop_words:  # Check if the word is not in the stop_words list
            return_list.append(x)  # If not, add it to the return_list
    return ' '.join(return_list)  # Join the words in return_list back into a string and return it


import re

def remove_foreign(x):
    """
    Remove foreign characters from a given string.

    This function takes a string as input and removes any characters that are not
    part of the Devanagari script, which is commonly used for writing languages like Hindi, Sanskrit, etc.
    
    Args:
        x (str): The input string from which foreign characters are to be removed.

    Returns:
        str: The input string with foreign characters removed.
    
    Raises:
        None
    
    """
    string = x.split(' ')  # Split the input string into a list of words
    y = [(re.compile(r'[\u0901-\u0939\u093C-\u094D\u0950-\u0954\u0958-\u0963\u097B-\u097F]')).findall(s) for s in string]
    z = [''.join(s) for s in y]  # Join the characters found in each word back into a string
    w = ' '.join(z)  # Join the resulting strings back into a single string
    return w  # Return the string with foreign characters removed


from tqdm import tqdm
@runtime
@memory_location
def pre(source, destination):
    """
    Preprocess a source text file and save the result to a destination file.

    This function reads text from the source file, removes foreign characters
    and stop words, and writes the preprocessed text to the destination file.
    It also updates a progress bar to show the processing progress.

    Args:
        source (str): The path to the source text file.
        destination (str): The path to save the preprocessed text file.

    Returns:
        None

    Raises:
        None

    """
    line_count = 0
    with open(source, 'r', encoding='utf-8') as input:
        with open(destination, 'w', encoding='utf-8') as output:
            bar = tqdm(total=5000000)  # Assuming 5000000 is the total number of lines
            for line in input:
                string = line.replace('\n', '')
                string = remove_foreign(string)
                string = remove_stop_words(string)
                output.write(string)
                line_count += 1
                if line_count % 10000 == 0:
                    bar.update(10000)  # Update progress bar every 10000 lines
            bar.close()



In [ ]:
pre('Hindi_Corpus.txt', 'ProcessedCorpus.txt')

In [7]:
from collections import Counter

min_vocab=0

def list_of_words():
	with open('ProcessedCorpus.txt', 'r', encoding='utf-8') as r:
		text = r.read().split()
	word_counts = Counter(text)
	text = [word for word in tqdm(text) if word_counts[word] > min_vocab]  
	return text

def gen_distinct_vocab():
	with open('ProcessedCorpus.txt', 'r', encoding='utf-8') as r:
		text = r.read().split()
		counter = Counter(text)
		reduced_vocab = [item for item, count in counter.items() if count > min_vocab]
		text_set = set(reduced_vocab)
		global vocabulary_size
		vocabulary_size = len(text_set)
	return list(text_set)


In [8]:

global words_list
words_list=list_of_words()

100%|██████████| 3944/3944 [00:00<00:00, 8034159.77it/s]


In [9]:
vocab=gen_distinct_vocab()
vocabulary_size = len(vocab)
tokens = word_tokenize(' '.join(words_list))
grams = ngrams(tokens, 6)

In [10]:
print("Number of tokens is: ", len(tokens))
print("Size of the vocabulary: ", vocabulary_size)


Number of tokens is:  3944
Size of the vocabulary:  1774


In [11]:

matrix = np.zeros((len(vocab),len(vocab)),dtype=np.int64)

@runtime
@memory_location
def create_matrix():

    for x in tqdm(grams):
    
        
        if x[0] in vocab:
            if x[1] in vocab:
                matrix[vocab.index(x[0])][vocab.index(x[1])] += 5
            if x[2] in vocab:
                matrix[vocab.index(x[0])][vocab.index(x[2])] += 4
            if x[3] in vocab:
                matrix[vocab.index(x[0])][vocab.index(x[3])] += 3
            if x[4] in vocab:
                matrix[vocab.index(x[0])][vocab.index(x[4])] += 2
            if x[5] in vocab:
                matrix[vocab.index(x[0])][vocab.index(x[5])] += 1
        if x[5] in vocab:
            if x[1] in vocab:
                matrix[vocab.index(x[5])][vocab.index(x[1])] += 2
            if x[2] in vocab:
                matrix[vocab.index(x[5])][vocab.index(x[2])] += 3
            if x[3] in vocab:
                matrix[vocab.index(x[5])][vocab.index(x[3])] += 4
            if x[4] in vocab:
                matrix[vocab.index(x[5])][vocab.index(x[4])] += 5
            if x[0] in vocab:
                matrix[vocab.index(x[5])][vocab.index(x[0])] += 1

        
create_matrix()


Location of create_matrix: 125689201869824


3939it [00:00, 6034.85it/s]

Time taken by wrapper: 0.6548962593078613 seconds


In [12]:
import math
from tqdm import tqdm

@runtime
@memory_location
def calculate_probability(matrix):
	
	row_sums = np.sum(matrix, axis=1)
	col_sums = np.sum(matrix, axis=0)

	N = np.sum(matrix)

	if(N==0):
		return 0,0

	global pi,pj

	pi = row_sums / N
	pj = col_sums / N

 
 
@runtime
@memory_location
def gen_ppmi_matrix(matrix):
	co_occurrence_matrix=matrix
 
	ppmi = np.zeros((len(vocab), len(vocab)))

	row_sums = np.sum(co_occurrence_matrix, axis=1)
	for i in tqdm(range(len(vocab))):
		for j in range(len(vocab)):

			if (row_sums[i] == 0 or co_occurrence_matrix[i][j] == 0):
				ppmi[i][j] = 0
				continue
			pij = ((co_occurrence_matrix[i][j]))/row_sums[i]
			ppmi[i][j] = max(0, math.log2(pij/(pi[i]*pj[j])))

	return ppmi


In [13]:
calculate_probability(matrix)
ppmi_matrix=gen_ppmi_matrix(matrix)


Location of calculate_probability: 125689201870304
Time taken by wrapper: 0.004347801208496094 seconds
Location of gen_ppmi_matrix: 125689201870464


100%|██████████| 1774/1774 [00:00<00:00, 1777.13it/s]

Time taken by wrapper: 1.001314640045166 seconds


In [14]:
from collections import Counter

counter = Counter(list_of_words())
common_elements = counter.most_common(10)
most_common_elements=[a for (a,_) in common_elements]

100%|██████████| 3944/3944 [00:00<00:00, 4113985.32it/s]


In [15]:
import math

def insert_into_sorted_list(sorted_list, element):
    index = 0
    while index < len(sorted_list) and sorted_list[index][1] > element[1]:
        index += 1
    sorted_list.insert(index, element)
    return sorted_list[:-1]

def cosine(a,b):
        mag_a = math.sqrt(sum(component ** 2 for component in a))
        mag_b = math.sqrt(sum(component ** 2 for component in b))
        dot_product = sum(ai * bi for ai, bi in zip(a, b))
        if mag_a == 0 or mag_b ==0:
            return 0
        else:
            return dot_product/(mag_a*mag_b)

def find_nearest_neighbor_of_noun(index):
    l=[(vocab[0],(cosine(ppmi_matrix[index], ppmi_matrix[0]))) for _ in range(11)]
    for i in tqdm(range(1,len(vocab))):
        if i!=index:
            if cosine(ppmi_matrix[index],ppmi_matrix[i])>l[10][1]:
                l=insert_into_sorted_list(l,(vocab[i],(cosine(ppmi_matrix[index], ppmi_matrix[i]))))
    return l

nearest_neighbour_dict={}

@runtime
@memory_location
def nearest_seq():
    for x in tqdm(most_common_elements):
        nearest_neighbour_dict[x]=find_nearest_neighbor_of_noun(vocab.index(x))

nearest_seq()

print(nearest_neighbour_dict)


Location of nearest_seq: 125689201872224


100%|██████████| 10/10 [00:09<00:00,  1.03it/s]

Time taken by wrapper: 9.753339290618896 seconds
{'हिन्दी': [('भाषा', 0.3828333162560839), ('भारत', 0.37609673574581554), ('रूप', 0.3477976068694602), ('भारतीय', 0.3284881293616273), ('उर्दू', 0.3280451644345689), ('प्रयोग', 0.305675297015081), ('भाषाओं', 0.28105872329174497), ('लोगों', 0.2688475790890009), ('बोली', 0.2665104863461141), ('आधुनिक', 0.2558838703714098), ('संख्या', 0.2550382375749371)], 'ॐ': [('नमः', 0.8482123197195954), ('विनायकाय', 0.6552663538005526), ('सिद्धि', 0.6499178515580774), ('समर्पयामि', 0.6134782932775708), ('श्री', 0.4270907762080836), ('पूजयामि', 0.3958441800594761), ('महा', 0.38962230677027127), ('दन्ताय', 0.35423902377597016), ('राजाय', 0.3495139749613974), ('विघ्न', 0.3485530702997846), ('पुष्पं', 0.3267200126202506)], 'भाषा': [('रूप', 0.42782447626726144), ('हिन्दी', 0.3828333162560839), ('शब्द', 0.29446673259386297), ('प्रयोग', 0.29271778384468583), ('हिन्दुस्तानी', 0.2876924021522872), ('उर्दू', 0.287402620434434), ('भारत', 0.2850153185172091), ('वाली

In [17]:
import multiprocessing
from tqdm import tqdm

n = vocabulary_size
p_mat = multiprocessing.Array('d', n*n)

def compile_grams(x,p_mat):
    leng=len(vocab)
    

    if x[0] in vocab:
        if x[1] in vocab:
            p_mat[vocab.index(x[0])*leng+vocab.index(x[1])] += 5
        if x[2] in vocab:
            p_mat[vocab.index(x[0])*leng+vocab.index(x[2])] += 4
        if x[3] in vocab:
            p_mat[vocab.index(x[0])*leng+vocab.index(x[3])] += 3
        if x[4] in vocab:
            p_mat[vocab.index(x[0])*leng+vocab.index(x[4])] += 2
        if x[5] in vocab:
            p_mat[vocab.index(x[0])*leng+vocab.index(x[5])] += 1
    if x[5] in vocab:
        if x[1] in vocab:
            p_mat[vocab.index(x[5])*leng+vocab.index(x[1])] += 2
        if x[2] in vocab:
            p_mat[vocab.index(x[5])*leng+vocab.index(x[2])] += 3
        if x[3] in vocab:
            p_mat[vocab.index(x[5])*leng+vocab.index(x[3])] += 4
        if x[4] in vocab:
            p_mat[vocab.index(x[5])*leng+vocab.index(x[4])] += 5
        if x[0] in vocab:
            p_mat[vocab.index(x[5])*leng+vocab.index(x[0])] += 1

@runtime
@memory_location
def create_matrix_p(p_mat):
    grams = ngrams(tokens,6)

    jobs=[]

    for x in tqdm(grams):
        p=multiprocessing.Process(target=compile_grams, args=(x,p_mat,))
        p.start()
        jobs.append(p)

    for job in jobs:
        job.join()

create_matrix_p(p_mat)

p_matrix = np.zeros((n,n),dtype=np.int64)

for i in range(n):
    for j in range(n):
        p_matrix[i][j]=p_mat[i*n+j]



Location of create_matrix_p: 125689200098208


3939it [00:11, 331.78it/s]


Time taken by wrapper: 11.890224695205688 seconds


In [19]:


neigh = multiprocessing.Manager().dict()

def p_insert_into_sorted_list(sorted_list, element):
	index = 0
	while index < len(sorted_list) and sorted_list[index][1] > element[1]:
		index += 1
	sorted_list.insert(index, element)
	return sorted_list[:-1]

def p_cosine(a,b):
		mag_a = math.sqrt(sum(component ** 2 for component in a))
		mag_b = math.sqrt(sum(component ** 2 for component in b))
		dot_product = sum(ai * bi for ai, bi in zip(a, b))
		if mag_a == 0 or mag_b ==0:
			return 0
		else:
			return dot_product/(mag_a*mag_b)

def p_find_nearest_neighbor_of_noun(index,neigh):
	l=[(vocab[0],(p_cosine(ppmi_matrix[index], ppmi_matrix[0]))) for _ in range(11)]
	for i in tqdm(range(1,n)):
		if i!=index:
			if p_cosine(ppmi_matrix[index],ppmi_matrix[i])>l[10][1]:
				l=p_insert_into_sorted_list(l,(vocab[i],(p_cosine(ppmi_matrix[index], ppmi_matrix[i]))))

	neigh[vocab[index]]=l

@runtime
@memory_location
def parallel_nearest():
	jobs=[]
	for x in (most_common_elements):
		p=multiprocessing.Process(target=p_find_nearest_neighbor_of_noun, args=(vocab.index(x),neigh,))
		p.start()
		jobs.append(p)

	for job in jobs:
		job.join()
		
parallel_nearest()

neighbour={}

for elem in most_common_elements:
	neighbour[elem]=neigh[elem]

for i in neighbour:
	print("Words similar to ",i," :")
	for j in neighbour[i]:
		print ("  ",j[0]," : ",j[1])

Location of parallel_nearest: 125689200098368


100%|██████████| 1773/1773 [00:01<00:00, 973.79it/s] 


Time taken by wrapper: 1.880664587020874 seconds
Words similar to  हिन्दी  :
   भाषा  :  0.3828333162560839
   भारत  :  0.37609673574581554
   रूप  :  0.3477976068694602
   भारतीय  :  0.3284881293616273
   उर्दू  :  0.3280451644345689
   प्रयोग  :  0.305675297015081
   भाषाओं  :  0.28105872329174497
   लोगों  :  0.2688475790890009
   बोली  :  0.2665104863461141
   आधुनिक  :  0.2558838703714098
   संख्या  :  0.2550382375749371
Words similar to  ॐ  :
   नमः  :  0.8482123197195954
   विनायकाय  :  0.6552663538005526
   सिद्धि  :  0.6499178515580774
   समर्पयामि  :  0.6134782932775708
   श्री  :  0.4270907762080836
   पूजयामि  :  0.3958441800594761
   महा  :  0.38962230677027127
   दन्ताय  :  0.35423902377597016
   राजाय  :  0.3495139749613974
   विघ्न  :  0.3485530702997846
   पुष्पं  :  0.3267200126202506
Words similar to  भाषा  :
   रूप  :  0.42782447626726144
   हिन्दी  :  0.3828333162560839
   शब्द  :  0.29446673259386297
   प्रयोग  :  0.29271778384468583
   हिन्दुस्तानी  :  0.28769240